In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
url = 'https://media.githubusercontent.com/media/buds-lab/building-data-genome-project-2/refs/heads/master/data/meters/cleaned/electricity_cleaned.csv'

data = pd.read_csv(url, parse_dates=['timestamp'], index_col='timestamp')

In [3]:
data.head()

,Panther_parking_Lorriane,Panther_lodging_Cora,Panther_office_Hannah,Panther_lodging_Hattie,Panther_education_Teofila,Panther_education_Jerome,Panther_retail_Felix,Panther_parking_Asia,Panther_education_Misty,Panther_retail_Gilbert,...,Cockatoo_public_Caleb,Cockatoo_education_Tyler,Cockatoo_public_Shad,Mouse_health_Buddy,Mouse_health_Modesto,Mouse_lodging_Vicente,Mouse_health_Justin,Mouse_health_Ileana,Mouse_health_Estela,Mouse_science_Micheal
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,123.200,727.575,69.200,8.8224,370.0870,10.0,282.9965,26.0,135.0,168.2243
2016-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,126.475,731.200,66.275,17.6449,737.8260,30.0,574.9265,51.0,265.0,336.4486
2016-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,127.825,724.675,64.675,17.6449,729.9255,30.0,570.2780,50.0,272.0,336.4486
2016-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,130.475,737.375,65.875,17.6449,722.2620,20.0,561.1470,52.0,276.0,336.4486
2016-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,129.675,721.150,66.275,17.6449,719.1665,30.0,564.3695,50.0,280.0,336.4486


In [4]:
data.shape

(17544, 1578)

In [5]:
data.isnull().sum().sort_values(ascending=False)

Hog_assembly_Arlie        17544
Hog_assembly_Una          17544
Hog_assembly_Marilynn     17544
Eagle_assembly_Noel       17544
Eagle_lodging_Garland     17544
                          ...  
Robin_lodging_Donna           0
Robin_office_Antonina         0
Robin_office_Zelma            0
Robin_education_Jasper        0
Robin_public_Carolina         0
Length: 1578, dtype: int64

Data Size and Importance:

If you have a large dataset with a wide range of data points, dropping a few columns might not be a significant problem. However, if the data is limited, or the missing columns contain crucial information, a higher threshold might be necessary.

Time Granularity:

- Yearly Data: Missing values in yearly data might not be as critical as in daily data because you're likely aggregating or averaging over longer periods. A higher percentage of missing values might be tolerated for yearly data.

- Monthly Data: Missing values are more noticeable in monthly data as you're looking at shorter time intervals. A lower percentage of missing values might be preferred.

- Daily Data: Missing daily values can significantly impact the analysis, especially for time series forecasting. The threshold for dropping should be low. 

In [7]:
# Calculate yearly missing value ratios
miss_ratio_y = data.resample('YE').apply(lambda x: x.isna().mean())

# Identify columns to drop based on yearly missing value threshold
cols_drop_y = miss_ratio_y.columns[
    (miss_ratio_y > 0.06).any()
].tolist()

miss_ratio_m = data.resample('ME').apply(lambda x: x.isna().mean())

cols_drop_m = miss_ratio_m.columns[
    (miss_ratio_m > 0.12).any()
].tolist()

miss_ratio_d = data.resample('D').apply(lambda x: x.isna().mean())

cols_drop_d = miss_ratio_d.columns[
    (miss_ratio_d > 0.55).any()
].tolist()


In [8]:
len(cols_drop_y), len(cols_drop_m), len(cols_drop_d)

(583, 698, 1188)

In [9]:
all_cols_to_drop = set(cols_drop_y + cols_drop_m + cols_drop_d)
cols_existing = data.columns.intersection(all_cols_to_drop)
data = data.drop(columns=cols_existing)

In [10]:
data.shape

(17544, 389)

In [11]:
data = data.ffill().bfill()

In [12]:
data.isnull().sum().sort_values(ascending=True).head(10)

Hog_assembly_Jasmine    0
Hog_lodging_Hal         0
Hog_office_Richelle     0
Hog_office_Vera         0
Hog_public_Crystal      0
Hog_public_Brad         0
Hog_education_Robert    0
Hog_education_Odell     0
Hog_office_Sherrie      0
Hog_assembly_Pedro      0
dtype: int64

In [13]:
data.duplicated().sum()

np.int64(0)